# Nos capítulos anteriores...

No primeiro arquivo fizemos uma análise exploratória dos dados dos experimentos e no segundo arquivo, analizamos os dados dos resultados dos experimentos. Está na hora e juntarmos esses dois conjutos de dados e utilizarmos técnicas de machine learning para tentar fazer preivisões de problemas. 
Mas antes, precisamos carregar novamente os bancos de dados.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') #ignora as mensagens de aviso

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [3]:
url_dados = 'https://github.com/alura-cursos/imersaodados3/blob/main/dados/dados_experimentos.zip?raw=true'
df = pd.read_csv(url_dados, compression = 'zip')
df.head()

,id,tratamento,tempo,dose,droga,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,...,c-60,c-61,c-62,c-63,c-64,c-65,c-66,c-67,c-68,c-69,c-70,c-71,c-72,c-73,c-74,c-75,c-76,c-77,c-78,c-79,c-80,c-81,c-82,c-83,c-84,c-85,c-86,c-87,c-88,c-89,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,com_droga,24,D1,b68db1d53,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,1.1830,0.1530,0.5574,-0.4015,0.1789,-0.6528,-0.7969,0.6342,0.1778,-0.3694,-0.5688,-1.1360,-1.1880,0.6940,0.4393,0.2664,0.1907,0.1628,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.3010,-0.1537,...,0.4805,0.4965,0.3680,0.8427,0.1042,0.1403,0.1758,1.2570,-0.5979,1.2250,-0.0553,0.7351,0.5810,0.9590,0.2427,0.0495,0.4141,0.8432,0.6162,-0.7318,1.2120,0.6362,-0.4427,0.1288,1.4840,0.1799,0.5367,-0.1111,-1.0120,0.6685,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,com_droga,72,D1,df89a8e5a,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,-1.1520,-0.4201,-0.0958,0.4590,0.0803,0.2250,0.5293,0.2839,-0.3494,0.2883,0.9449,-0.1646,-0.2657,-0.3372,0.3135,-0.4316,0.4773,0.2075,-0.4216,-0.1161,-0.0499,-0.2627,0.9959,-0.2483,0.2655,...,0.4083,0.0319,0.3905,0.7099,0.2912,0.4151,-0.2840,-0.3104,-0.6373,0.2887,-0.0765,0.2539,0.4443,0.5932,0.2031,0.7639,0.5499,-0.3322,-0.0977,0.4329,-0.2782,0.7827,0.5934,0.3402,0.1499,0.4420,0.9366,0.8193,-0.4236,0.3192,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,com_droga,48,D1,18bb41b2c,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,-0.4797,-0.5631,-0.0366,-1.8300,0.6057,-0.3278,0.6042,-0.3075,-0.1147,-0.0570,-0.0799,-0.8181,-1.5320,0.2307,0.4901,0.4780,-1.3970,4.6240,-0.0437,1.2870,-1.8530,0.6069,0.4290,0.1783,0.0018,...,-0.5477,-0.7576,-0.0444,0.1894,-0.0014,-2.3640,-0.4682,0.1210,-0.5177,-0.0604,0.1682,-0.4436,0.4963,0.1363,0.3335,0.9760,-0.0427,-0.1235,0.0959,0.0690,-0.9416,-0.7548,-0.1109,-0.6272,0.3019,0.1172,0.1093,-0.3113,0.3019,-0.0873,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,com_droga,48,D1,8c7f86626,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,-0.8269,-0.3584,-0.8511,-0.5844,-2.5690,0.8183,-0.0532,-0.8554,0.1160,-2.3520,2.1200,-1.1580,-0.7191,-0.8004,-1.4670,-0.0107,-0.8995,0.2406,-0.2479,-1.0890,-0.7575,0.0881,-2.7370,0.8745,0.5787,...,-2.1220,-0.3752,-2.3820,-3.7350,-2.9740,-1.4930,-1.6600,-3.1660,0.2816,-0.2990,-1.1870,-0.5044,-1.7750,-1.6120,-0.9215,-1.0810,-3.0520,-3.4470,-2.7740,-1.8460,-0.5568,-3.3960,-2.9510,-1.1550,-3.2620,-1.5390,-2.4600,-0.9417,-1.5550,0.2431,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,com_droga,72,D2,7cbed3131,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,-0.2219,-0.5121,-0.9577,1.1750,0.2042,0.1970,0.1244,-1.7090,-0.3543,-0.5160,-0.3330,-0.2685,0.7649,0.2057,1.3720,0.6835,0.8056,-0.3754,-1.2090,0.2965,-0.0712,0.6389,0.6674,-0.0783,1.1740,...,-0.2274,0.3215,0.1535,-0.4640,-0.5943,0.3973,0.1500,0.5178,0.5159,0.6091,0.1813,-0.4249,0.7832,0.6529,0.5648,0.4817,0.0587,0.5303,0.6376,-0.3966,-1.4950,-0.9625,-0.0541,0.6273,0.4563,0.0698,0.8134,0.1924,0.6054,-0.1824,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [4]:
#Renomeando as colunas, eliminando '-'
colunas = df.columns.to_numpy()  #cria um array com os nomes das colunas
for i in range(0,colunas.shape[0]):
   colunas[i] = colunas[i].replace('-','')  #faz um loop percorrendo os índices do array, trocando '-' por ''
df.columns = colunas  #renomeia as colunas
df.rename(columns= {'droga':'composto'}, inplace = True) #renomeia a coluna 'droga' para 'composto'
df.tail() #mostra os 5 últimos regristos

,id,tratamento,tempo,dose,composto,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18,g19,g20,g21,g22,g23,g24,g25,g26,g27,g28,g29,g30,g31,g32,g33,g34,...,c60,c61,c62,c63,c64,c65,c66,c67,c68,c69,c70,c71,c72,c73,c74,c75,c76,c77,c78,c79,c80,c81,c82,c83,c84,c85,c86,c87,c88,c89,c90,c91,c92,c93,c94,c95,c96,c97,c98,c99
23809,id_fffb1ceed,com_droga,24,D2,df1d0a5a1,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,0.3055,-0.4726,0.1269,0.2531,0.1730,-0.4532,-1.0790,0.2474,-0.4550,0.3588,0.1600,-0.7362,-0.1103,0.8550,-0.4139,0.5541,0.2310,-0.5573,-0.4397,-0.9260,-0.2424,-0.6686,0.2326,0.6456,0.0136,-0.5141,-0.6320,0.7166,...,0.0807,0.4024,-0.0895,0.4420,0.9641,-0.1846,-0.0014,0.3154,-0.2071,-0.6158,-0.2977,0.0992,0.6838,0.5259,0.7882,0.3119,-0.7697,0.2203,-1.0710,0.5979,0.0848,-0.2555,0.6293,1.1660,0.3329,0.2754,0.4108,-0.1252,-0.2340,0.2267,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
23810,id_fffb70c0c,com_droga,24,D2,ecf3b6b74,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,-0.2252,-0.5565,0.5112,0.6727,-0.1851,2.8650,-0.2140,-0.6153,0.8362,0.5584,-0.2589,0.1292,0.0148,0.0949,-0.2182,-0.9235,0.0749,-1.5910,-0.8359,-0.9217,0.3013,0.1716,0.0880,0.1842,0.1835,0.5436,-0.0533,-0.0491,0.9543,...,0.1410,1.2640,-0.8663,0.8129,-0.1514,-0.4652,-0.7390,-1.3270,0.9925,1.0570,-0.3355,-0.2555,0.8219,0.6995,-0.2942,0.2408,-0.7781,-0.0929,-0.0329,0.0781,-1.4440,-0.0921,0.3188,-1.1080,0.4895,-0.2144,1.0960,0.3335,0.4444,-1.1130,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
23811,id_fffc1c3f4,com_controle,48,D2,cacb2b860,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,-0.2541,0.1745,-0.0340,0.4865,-0.1854,0.0716,0.1729,-0.0434,0.1542,-0.2192,-0.0302,-0.4218,0.4057,-0.5372,0.1521,-0.2651,0.2310,-0.8101,0.4943,0.6905,-0.3720,-1.4110,0.4516,1.2300,-0.1949,-1.3280,-0.4276,-0.0040,-0.3086,...,0.6845,0.7127,0.7294,0.4718,-0.2020,0.2783,0.4934,0.4144,0.5449,1.4690,-0.6142,0.6068,0.3434,0.9880,-0.0468,-0.1882,-0.0087,-0.0356,0.5718,0.4971,0.1057,0.2111,0.6992,0.0708,0.6169,0.2248,0.5994,0.2689,0.0305,1.2320,0.5409,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808
23812,id_fffcb9e7c,com_droga,24,D1,8b87a7a83,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,0.0323,0.0463,0.4299,-0.7985,0.5742,0.1421,2.2700,0.2046,0.5363,-1.7330,0.1450,0.6097,0.2024,0.9865,-0.7805,0.9608,0.3440,2.7650,0.4925,0.6698,0.2374,-0.3372,0.8771,-2.6560,-0.2000,-0.2043,0.6797,-0.0248,-0.0927,...,0.3360,-0.6136,0.5011,0.9261,0.4419,0.0295,0.4220,0.4677,-0.1184,0.4524,0.4135,0.1356,-0.5801,0.0411,1.0240,1.0340,-0.0270,-0.4194,0.7403,-0.6793,-0.1423,0.7307,0.7946,-0.0650,0.9038,0.2324,0.9676,1.0940,0.4290,0.5187,-0.1105,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740
23813,id_ffffdd77b,com_droga,72,D1,972f41291,-0.8598,1.0240,-0.1361,0.7952,-0.3611,-3.6750,-1.2420,0.9146,3.0790,1.2460,1.9460,1.4370,2.9780,2.2370,-0.6818,0.6870,-1.1060,0.0182,-0.9247,-0.0738,-0.1919,-0.7722,-1.4050,-1.0050,-1.1170,-0.5293,-1.1720,-0.2885,0.1599,-0.4250,0.3591,-0.1420,-0.9530,-0.2005,-1.8340,...,-6.9170,-1.8640,-2.5090,-4.7130,-1.7250,-3.8650,-3.0800,-4.1530,-1.2030,-1.1690,-4.1460,-1.2670,-1.1300,-2.4390,0.1591,-2.2490,-2.5860,-1.9520,-2.1810,-4.6690,-3.9450,-2.9560,-2.7930,-2.1560,-2.4100,-1.8190,-3.3480,-0.1414,-2.6430,-2.5810,-3.3890,-1.7450,-6.6300,-4.0950,-7.3860,-1.4160,-3.5770,-0.4775,-2.1500,-4.2520


In [5]:
df_resultados = pd.read_csv('https://github.com/alura-cursos/imersaodados3/blob/main/dados/dados_resultados.csv?raw=true')
df_resultados

,id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,...,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23810,id_fffb70c0c,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23811,id_fffc1c3f4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23812,id_fffcb9e7c,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
df_resultados['n_moa'] = df_resultados.drop('id', axis=1).sum(axis=1)

In [7]:
df_resultados['ativo_moa'] = (df_resultados['n_moa'] != 0)*1
df_resultados.head()

,id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,...,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,n_moa,ativo_moa
0,id_000644bb2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1,id_000779bfc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1
3,id_0015fd391,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [8]:
df_total = pd.merge(df, df_resultados[['id','n_moa', 'ativo_moa']], on='id')
df_total.head()

,id,tratamento,tempo,dose,composto,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18,g19,g20,g21,g22,g23,g24,g25,g26,g27,g28,g29,g30,g31,g32,g33,g34,...,c62,c63,c64,c65,c66,c67,c68,c69,c70,c71,c72,c73,c74,c75,c76,c77,c78,c79,c80,c81,c82,c83,c84,c85,c86,c87,c88,c89,c90,c91,c92,c93,c94,c95,c96,c97,c98,c99,n_moa,ativo_moa
0,id_000644bb2,com_droga,24,D1,b68db1d53,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,1.1830,0.1530,0.5574,-0.4015,0.1789,-0.6528,-0.7969,0.6342,0.1778,-0.3694,-0.5688,-1.1360,-1.1880,0.6940,0.4393,0.2664,0.1907,0.1628,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.3010,-0.1537,...,0.3680,0.8427,0.1042,0.1403,0.1758,1.2570,-0.5979,1.2250,-0.0553,0.7351,0.5810,0.9590,0.2427,0.0495,0.4141,0.8432,0.6162,-0.7318,1.2120,0.6362,-0.4427,0.1288,1.4840,0.1799,0.5367,-0.1111,-1.0120,0.6685,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176,1,1
1,id_000779bfc,com_droga,72,D1,df89a8e5a,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,-1.1520,-0.4201,-0.0958,0.4590,0.0803,0.2250,0.5293,0.2839,-0.3494,0.2883,0.9449,-0.1646,-0.2657,-0.3372,0.3135,-0.4316,0.4773,0.2075,-0.4216,-0.1161,-0.0499,-0.2627,0.9959,-0.2483,0.2655,...,0.3905,0.7099,0.2912,0.4151,-0.2840,-0.3104,-0.6373,0.2887,-0.0765,0.2539,0.4443,0.5932,0.2031,0.7639,0.5499,-0.3322,-0.0977,0.4329,-0.2782,0.7827,0.5934,0.3402,0.1499,0.4420,0.9366,0.8193,-0.4236,0.3192,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371,0,0
2,id_000a6266a,com_droga,48,D1,18bb41b2c,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,-0.4797,-0.5631,-0.0366,-1.8300,0.6057,-0.3278,0.6042,-0.3075,-0.1147,-0.0570,-0.0799,-0.8181,-1.5320,0.2307,0.4901,0.4780,-1.3970,4.6240,-0.0437,1.2870,-1.8530,0.6069,0.4290,0.1783,0.0018,...,-0.0444,0.1894,-0.0014,-2.3640,-0.4682,0.1210,-0.5177,-0.0604,0.1682,-0.4436,0.4963,0.1363,0.3335,0.9760,-0.0427,-0.1235,0.0959,0.0690,-0.9416,-0.7548,-0.1109,-0.6272,0.3019,0.1172,0.1093,-0.3113,0.3019,-0.0873,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931,3,1
3,id_0015fd391,com_droga,48,D1,8c7f86626,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,-0.8269,-0.3584,-0.8511,-0.5844,-2.5690,0.8183,-0.0532,-0.8554,0.1160,-2.3520,2.1200,-1.1580,-0.7191,-0.8004,-1.4670,-0.0107,-0.8995,0.2406,-0.2479,-1.0890,-0.7575,0.0881,-2.7370,0.8745,0.5787,...,-2.3820,-3.7350,-2.9740,-1.4930,-1.6600,-3.1660,0.2816,-0.2990,-1.1870,-0.5044,-1.7750,-1.6120,-0.9215,-1.0810,-3.0520,-3.4470,-2.7740,-1.8460,-0.5568,-3.3960,-2.9510,-1.1550,-3.2620,-1.5390,-2.4600,-0.9417,-1.5550,0.2431,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154,0,0
4,id_001626bd3,com_droga,72,D2,7cbed3131,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,-0.2219,-0.5121,-0.9577,1.1750,0.2042,0.1970,0.1244,-1.7090,-0.3543,-0.5160,-0.3330,-0.2685,0.7649,0.2057,1.3720,0.6835,0.8056,-0.3754,-1.2090,0.2965,-0.0712,0.6389,0.6674,-0.0783,1.1740,...,0.1535,-0.4640,-0.5943,0.3973,0.1500,0.5178,0.5159,0.6091,0.1813,-0.4249,0.7832,0.6529,0.5648,0.4817,0.0587,0.5303,0.6376,-0.3966,-1.4950,-0.9625,-0.0541,0.6273,0.4563,0.0698,0.8134,0.1924,0.6054,-0.1824,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125,1,1


In [9]:
df_total['Eh controle'] = (df_total['tratamento'] == 'com_controle')*1
df_total['Tempo eh 24'] = (df_total['tempo'] == 24)*1
df_total['Tempo eh 48'] = (df_total['tempo'] == 48)*1
df_total['Tempo eh 72'] = (df_total['tempo'] == 72)*1

# Máquinas preditivas

## Conceitos importantes

Antes iniciarmos a construção dos modelos, é necessário explicar alguns conceitos que surgirão em nossa análise.

A acurácia ('score', definida como total de acertos / total de amostras) não será a única métrica que utilizaremos para analisar a performace dos nossos modelos. Dependendo dos aspectos do problema de negócio, podemos estar interessandos em outras métricas.

Uma boa maneira de começar a melhorar a análise da performace do modelo é verificar sua matriz de confusão ('confusion matrix'). Nas linhas dessa matriz temos os resultados reais e nas colunas temos os resultados previstos pelo modelo, de acordo com cada classe. 


Em problemas de classificação binária, será uma matrix 2x2. 

Nas células dessa matriz, temos 4 informações importantes:
- VP = número de verdadeiros positivos (modelo previu que era positivo e de fato era positivo)
- VN = número de verdadeiros negativos (modelo previu que era negativo e de fato era negativo)
- FP = número de falsos positivos (modelo previu que era positivo, mas na verdade era negativo)
- FN = número de falsos negativos (modelo previu que era negativo, mas na verdade era positivo)

Dependendo do nosso problema, talvez queiramos um modelo com baixa proporção de falsos negativos, ou talvez uma baixa proporção de falsos positivos. 

Temos métricas que medem isso:
- precisão de positos = VP/(VP+FP)
- precisão de negativos = VN/(VN+FN)
- recall (ou revocação) de positivos = VP/(VP+FN)
- recall (ou revocação) de negativos = VN/(VN+FP)
- F1 = média harmônica entre presicão e revocação

## Problema 1: Prevendo se o composto ativa algum MOA

Como primeiro exemplo, vamos constuir máquinas que prevem se um composto ativa pelo menos um MOA (sem especificar qual é o MOA). Compostos que não ativam nenhum MOA provavelmente não são interessantes. 
Nesse caso, não queremos correr o risco de prever que um composto ativa, porém na verdade não ativa. Ou seja, falsos positivos não são bem-vindos nesse problema. Logo estamos interessando em modelos com alta precisão para positivos. 

Para começar, devemos separar, dentre as colunas do nosso data frame, aquelas que servirão para alimentar as máquinas preditivas e aquela que será a informação a ser prevista. 
Nesse primeiro momento, como variável a ser prevista, utilizaremos y = ativo_moa, isto é, vamos prever se o composto irá ou não ativar algum moa. 
Como parâmetros na variável x, iremos passar dados de dose, tempo, as expressões gênicas e as viabilidades celuares. 

Importante lembrar que já observamos que as viabilidades celuares são variáveis com alta correção entre si. Talvez seja então interessante não considerar todas elas no modelo preditivo. Como já vimos na Aula 5 que as máquinas construídas lá não obtivaram um score muito bom e foram utilizadas todas as viabilidades celuares, nos meus modelos decidi utilizar apenas c0 e ver se os resultados são melhores. 

In [ ]:
y = df_total['ativo_moa']
x_aux = df_total.drop(['id', 'tratamento', 'tempo','dose', 'composto', 'ativo_moa'], axis = 1)
x = x_aux.drop(x_aux.columns[773:873], axis = 1)

Agora iremos separar nossos conjuntos de treino e conjunto de teste.

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, test_size = 0.3, stratify = y)

Estamos prontos para começar a testar os modelos preditivos.
Vamos criar também um dataframe com informações sumarizadas das performaces de cada modelo, para que possamos compará-los no final. Criaremos listas que coletarão os resultados das métricas de análise e no fim do processo utilizaremos essas listas como colunas do nosso dataframe. Assim:

In [ ]:
lista_modelo = []
lista_score = []
lista_precision0 = [] 
lista_precision1 = []
lista_recall0 = []
lista_recall1 = []

### Regressão Logistica

In [ ]:
modelo = LogisticRegression()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  66.01819454163751 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)


[[1043 1767]
 [ 661 3674]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.61      0.37      0.46      2810
           1       0.68      0.85      0.75      4335

    accuracy                           0.66      7145
   macro avg       0.64      0.61      0.61      7145
weighted avg       0.65      0.66      0.64      7145



### DecisionTreeClassifier 

In [ ]:
modelo = DecisionTreeClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  62.02939118264521 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[1441 1369]
 [1344 2991]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.52      0.51      0.52      2810
           1       0.69      0.69      0.69      4335

    accuracy                           0.62      7145
   macro avg       0.60      0.60      0.60      7145
weighted avg       0.62      0.62      0.62      7145



### RandomFlorestClassifier

In [ ]:
modelo = RandomForestClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  67.30580825752274 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[ 783 2027]
 [ 309 4026]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.72      0.28      0.40      2810
           1       0.67      0.93      0.78      4335

    accuracy                           0.67      7145
   macro avg       0.69      0.60      0.59      7145
weighted avg       0.69      0.67      0.63      7145



### LGBMClassifier

In [ ]:
modelo = LGBMClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  68.24352694191742 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[ 808 2002]
 [ 267 4068]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.75      0.29      0.42      2810
           1       0.67      0.94      0.78      4335

    accuracy                           0.68      7145
   macro avg       0.71      0.61      0.60      7145
weighted avg       0.70      0.68      0.64      7145



### XGBClassifier

In [ ]:
modelo = XGBClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  68.99930020993702 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[ 630 2180]
 [  35 4300]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.95      0.22      0.36      2810
           1       0.66      0.99      0.80      4335

    accuracy                           0.69      7145
   macro avg       0.81      0.61      0.58      7145
weighted avg       0.78      0.69      0.63      7145



### GradientBoostingClassifier


In [ ]:
modelo = GradientBoostingClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  69.06927921623513 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[ 642 2168]
 [  42 4293]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.94      0.23      0.37      2810
           1       0.66      0.99      0.80      4335

    accuracy                           0.69      7145
   macro avg       0.80      0.61      0.58      7145
weighted avg       0.77      0.69      0.63      7145



### KNeighborsClassifier

In [ ]:
modelo = KNeighborsClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  61.371588523442966 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[1020 1790]
 [ 970 3365]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.51      0.36      0.42      2810
           1       0.65      0.78      0.71      4335

    accuracy                           0.61      7145
   macro avg       0.58      0.57      0.57      7145
weighted avg       0.60      0.61      0.60      7145



### Support Vector Machine

In [ ]:
modelo = SVC()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  68.45346396081176 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[ 702 2108]
 [ 146 4189]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.83      0.25      0.38      2810
           1       0.67      0.97      0.79      4335

    accuracy                           0.68      7145
   macro avg       0.75      0.61      0.59      7145
weighted avg       0.73      0.68      0.63      7145



### Resumo 

Vamos sumarizar essas informações sobre a performace das máquinas preditivas em uma tabela. 

In [ ]:
Tabela_Performace_Modelos = pd.DataFrame()
Tabela_Performace_Modelos['Modelo'] = lista_modelo
Tabela_Performace_Modelos['Score'] = lista_score
Tabela_Performace_Modelos['precision-0'] = lista_precision0
Tabela_Performace_Modelos['precision-1'] = lista_precision1
Tabela_Performace_Modelos['recall-0'] = lista_recall0
Tabela_Performace_Modelos['recall-1'] = lista_recall1

Tabela_Comparação_Modelos_Problema1 = Tabela_Performace_Modelos
Tabela_Comparação_Modelos_Problema1

,Modelo,Score,precision-0,precision-1,recall-0,recall-1
0,LogisticRegression,0.660182,0.612089,0.675244,0.371174,0.847520
1,DecisionTreeClassifier,0.620294,0.517415,0.686009,0.512811,0.689965
2,RandomForestClassifier,0.673058,0.717033,0.665125,0.278648,0.928720
3,LGBMClassifier,0.682435,0.751628,0.670181,0.287544,0.938408
4,XGBClassifier,0.689993,0.947368,0.663580,0.224199,0.991926
5,GradientBoostingClassifier,0.690693,0.938596,0.664448,0.228470,0.990311
6,KNeighborsClassifier,0.613716,0.512563,0.652764,0.362989,0.776240
7,SVC,0.684535,0.827830,0.665237,0.249822,0.966321


Percebemos então que a acurácia dos modelos não é boa. Mas como já explicado, estamos também muito interessados na precisão dos resultados positivos, o que também não são bons, nunca chegando a 70%. 
Levando em conta a acurácia e a precisão, concluímos que nenhum dos modelos é satisfatório para resolver o Problema 1. O que faz sentindo, pois perguntar se ativa algum moa parece algo bastante vago. 
Está claro então que precisamos melhorar a pergunta que queremos responder utilizando a máquina preditiva. Vamos mudar a abordagem.

## Problema 2: Prevendo se o composto ativa 2 ou mais MOAs

No Problema 1 estávamos interessando em saber se um composto ativa pelo menos 1 MOA. O Problema 2 é similar: queremos saber se um composto, com base em suas expressões gênicas, ativa 2 ou mais MOA. Ainda é uma pergunta vaga, mas veremos que os resultados são mais animadores. 

Vamos começar definindo a nossa variável alvo:

In [ ]:
df_total['ativa_2moa'] = (df_total['n_moa'] > 1)*1

Vejamos como é a distrubuição de valores dessa variável:

In [ ]:
df_total['ativa_2moa'].value_counts()

0    21899
1     1915
Name: ativa_2moa, dtype: int64

Observamos então que os dados estão bem desbalanceados. Isso pode afetar o desepenho dos modelos preditivos. Nesse tipo de cenário, é razoável esperar uma acurácia muito alta. Por esse motivo, mais do que nunca, é crucial considerar também a precisão de positivos. 

In [ ]:
y = df_total['ativa_2moa']
x_aux = df_total.drop(['id', 'tratamento', 'tempo','dose', 'composto', 'ativo_moa', 'ativa_2moa'], axis = 1)
x = x_aux.drop(x_aux.columns[773:873], axis = 1)

Agora iremos separar nossos conjuntos de treino e conjunto de teste.

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, test_size = 0.3, stratify = y)

Como antes, vamos criar também um dataframe com informações sumarizadas das performaces de cada modelo:

In [ ]:
lista_modelo = []
lista_score = []
lista_precision0 = [] 
lista_precision1 = []
lista_recall0 = []
lista_recall1 = []

### Regressão Logistica

In [ ]:
modelo = LogisticRegression()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  94.61161651504548 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6527   43]
 [ 342  233]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      6570
           1       0.84      0.41      0.55       575

    accuracy                           0.95      7145
   macro avg       0.90      0.70      0.76      7145
weighted avg       0.94      0.95      0.94      7145



### DecisionTreeClassifier 

In [ ]:
modelo = DecisionTreeClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  89.58712386284114 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6158  412]
 [ 332  243]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.95      0.94      0.94      6570
           1       0.37      0.42      0.40       575

    accuracy                           0.90      7145
   macro avg       0.66      0.68      0.67      7145
weighted avg       0.90      0.90      0.90      7145



### RandomFlorestClassifier

In [ ]:
modelo = RandomForestClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  94.9335199440168 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6562    8]
 [ 354  221]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6570
           1       0.97      0.38      0.55       575

    accuracy                           0.95      7145
   macro avg       0.96      0.69      0.76      7145
weighted avg       0.95      0.95      0.94      7145



### LGBMClassifier

In [ ]:
modelo = LGBMClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  95.01749475157453 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6563    7]
 [ 349  226]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6570
           1       0.97      0.39      0.56       575

    accuracy                           0.95      7145
   macro avg       0.96      0.70      0.77      7145
weighted avg       0.95      0.95      0.94      7145



### XGBClassifier

In [ ]:
modelo = XGBClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  94.96151154653603 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6563    7]
 [ 353  222]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6570
           1       0.97      0.39      0.55       575

    accuracy                           0.95      7145
   macro avg       0.96      0.69      0.76      7145
weighted avg       0.95      0.95      0.94      7145



### GradientBoostingClassifier


In [ ]:
modelo = GradientBoostingClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  94.77956613016096 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6551   19]
 [ 354  221]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6570
           1       0.92      0.38      0.54       575

    accuracy                           0.95      7145
   macro avg       0.93      0.69      0.76      7145
weighted avg       0.95      0.95      0.94      7145



### KNeighborsClassifier

In [ ]:
modelo = KNeighborsClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  94.6815955213436 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6542   28]
 [ 352  223]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6570
           1       0.89      0.39      0.54       575

    accuracy                           0.95      7145
   macro avg       0.92      0.69      0.76      7145
weighted avg       0.94      0.95      0.94      7145



### Support Vector Machine

In [ ]:
modelo = SVC()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  94.63960811756473 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6563    7]
 [ 376  199]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6570
           1       0.97      0.35      0.51       575

    accuracy                           0.95      7145
   macro avg       0.96      0.67      0.74      7145
weighted avg       0.95      0.95      0.93      7145



### Resumo

In [ ]:
Tabela_Performace_Modelos = pd.DataFrame()
Tabela_Performace_Modelos['Modelo'] = lista_modelo
Tabela_Performace_Modelos['Score'] = lista_score
Tabela_Performace_Modelos['precision-0'] = lista_precision0
Tabela_Performace_Modelos['precision-1'] = lista_precision1
Tabela_Performace_Modelos['recall-0'] = lista_recall0
Tabela_Performace_Modelos['recall-1'] = lista_recall1

Tabela_Comparação_Modelos_Problema2 = Tabela_Performace_Modelos
Tabela_Comparação_Modelos_Problema2

,Modelo,Score,precision-0,precision-1,recall-0,recall-1
0,LogisticRegression,0.946116,0.950211,0.844203,0.993455,0.405217
1,DecisionTreeClassifier,0.895871,0.948844,0.370992,0.937291,0.422609
2,RandomForestClassifier,0.949335,0.948814,0.965066,0.998782,0.384348
3,LGBMClassifier,0.950175,0.949508,0.969957,0.998935,0.393043
4,XGBClassifier,0.949615,0.948959,0.969432,0.998935,0.386087
5,GradientBoostingClassifier,0.947796,0.948733,0.920833,0.997108,0.384348
6,KNeighborsClassifier,0.946816,0.948941,0.888446,0.995738,0.387826
7,SVC,0.946396,0.945814,0.966019,0.998935,0.346087


Vemos que para o Problema 2, a acurácia de todos os modelos foi bastante alta, acima de 94%, como era de se esperar. Por isso, damos atenção à métrica precisão e notamos que também estão razoáveis. 
Com excessão do modelo de árvore de decisão, todos os outros modelos tiveram uma precisão acima de 84%. Alguns obtiveram precisão acima de 96%, o que é impressionante. 

Porém, uma métrica tão boa assim não vem de graça. É razoável esperar que outra métricas sejam afetadas negativamente. Vemos que em todos os modelos, o recall de positivos é baixo, sempre menor que 45%. 
Ou seja, existe um número proporcionalmente grande de falsos negativos. No nosso problema de negócios, isso significa que haverá um número grande de compostos cujo modelo prevê que não ativará os MOAs, mas na verdade ele ativará. Estamos perdendo oportunidades. 

Mas vale frisar novamente que uma alta precisão para positivos significa que temos uma proporção baixa de falsos positivos. Ou seja, estamos correndo um risco menor de aprovar um composto que não funcione (ou funcione de uma maneira que não é a desejada). 
Fica a cargo do tomador de decisão escolher qual cenário é preferível. 

## Problema 3: Prevendo se o composto ativa um MOA específico

No Problema 1 estávamos interessando em saber se um composto ativa pelo menos 1 MOA. No Problema 3 estamos interessados em saber se o composto ativa um MOA de nossa escolha. Biologicamente, esse problema faz bastante sentido e é parece ser mais relevante. 



Podemos ver esse problema sob duas perspectivas:
 

1.   queremos garantir que o composto ative o MOA, pois é benéfico para o paciente
2.   queremos garantir que o composto não ative o MOA, pois seria maléfico para o paciente



No primeiro cenário, queremos uma baixa proporção de falsos positivos, ou seja, uma precisão alta para positivos. No segundo cenário, queremos uma proporção baixa para falsos negativos, ou seja, uma precisão alta para negativos.

Para efeito de exemplificação, vamos escolher o MOA proteasome_inhibitor. Esse é o MOA que mais foi ativado, com 762 ativações. 

Para realizarmos isso, vamos precisar criar um novo data frame, utilizando o merge. 

In [ ]:
df_moa_ativado = pd.merge(df.iloc[:,0:778], df_resultados[['id','proteasome_inhibitor']], on='id')
df_moa_ativado

In [ ]:
df_moa_ativado['Eh controle'] = (df_moa_ativado['tratamento'] == 'com_controle')*1
df_moa_ativado['Tempo eh 24'] = (df_moa_ativado['tempo'] == 24)*1
df_moa_ativado['Tempo eh 48'] = (df_moa_ativado['tempo'] == 48)*1
df_moa_ativado['Tempo eh 72'] = (df_moa_ativado['tempo'] == 72)*1

Criando as variáveis:

In [ ]:
y = df_moa_ativado['proteasome_inhibitor']
x = df_moa_ativado.drop(['id', 'tratamento','composto' ,'dose', 'tempo', 'proteasome_inhibitor'], axis = 1)

Agora iremos separar nossos conjuntos de treino e conjunto de teste.

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, test_size = 0.3, stratify = y)

Estamos prontos para começar a testar os modelos preditivos.
Como antes, vamos criar também um dataframe com informações sumarizadas das performaces de cada modelo. Assim:

In [ ]:
lista_modelo = []
lista_score = []
lista_precision0 = [] 
lista_precision1 = []
lista_recall0 = []
lista_recall1 = []

### Regressão Logistica

In [ ]:
modelo = LogisticRegression()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.95801259622114 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6925    2]
 [   1  217]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6927
           1       0.99      1.00      0.99       218

    accuracy                           1.00      7145
   macro avg       1.00      1.00      1.00      7145
weighted avg       1.00      1.00      1.00      7145



### DecisionTreeClassifier 

In [ ]:
modelo = DecisionTreeClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.72008397480757 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6918    9]
 [  11  207]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6927
           1       0.96      0.95      0.95       218

    accuracy                           1.00      7145
   macro avg       0.98      0.97      0.98      7145
weighted avg       1.00      1.00      1.00      7145



### RandomFlorestClassifier

In [ ]:
modelo = RandomForestClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.94401679496151 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6924    3]
 [   1  217]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6927
           1       0.99      1.00      0.99       218

    accuracy                           1.00      7145
   macro avg       0.99      1.00      1.00      7145
weighted avg       1.00      1.00      1.00      7145



### LGBMClassifier

In [ ]:
modelo = LGBMClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.91602519244228 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6924    3]
 [   3  215]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6927
           1       0.99      0.99      0.99       218

    accuracy                           1.00      7145
   macro avg       0.99      0.99      0.99      7145
weighted avg       1.00      1.00      1.00      7145



### XGBClassifier

In [ ]:
modelo = XGBClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.91602519244228 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6924    3]
 [   3  215]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6927
           1       0.99      0.99      0.99       218

    accuracy                           1.00      7145
   macro avg       0.99      0.99      0.99      7145
weighted avg       1.00      1.00      1.00      7145



### GradientBoostingClassifier


In [ ]:
modelo = GradientBoostingClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.88803358992303 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6923    4]
 [   4  214]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6927
           1       0.98      0.98      0.98       218

    accuracy                           1.00      7145
   macro avg       0.99      0.99      0.99      7145
weighted avg       1.00      1.00      1.00      7145



### KNeighborsClassifier

In [ ]:
modelo = KNeighborsClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.76207137858643 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6911   16]
 [   1  217]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6927
           1       0.93      1.00      0.96       218

    accuracy                           1.00      7145
   macro avg       0.97      1.00      0.98      7145
weighted avg       1.00      1.00      1.00      7145



### Support Vector Machine

In [ ]:
modelo = SVC()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  99.65010496850944 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6922    5]
 [  20  198]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6927
           1       0.98      0.91      0.94       218

    accuracy                           1.00      7145
   macro avg       0.99      0.95      0.97      7145
weighted avg       1.00      1.00      1.00      7145



### Resumo

In [ ]:
Tabela_Performace_Modelos = pd.DataFrame()
Tabela_Performace_Modelos['Modelo'] = lista_modelo
Tabela_Performace_Modelos['Score'] = lista_score
Tabela_Performace_Modelos['precision-0'] = lista_precision0
Tabela_Performace_Modelos['precision-1'] = lista_precision1
Tabela_Performace_Modelos['recall-0'] = lista_recall0
Tabela_Performace_Modelos['recall-1'] = lista_recall1

Tabela_Comparação_Modelos_Problema3 = Tabela_Performace_Modelos
Tabela_Comparação_Modelos_Problema3

,Modelo,Score,precision-0,precision-1,recall-0,recall-1
0,LogisticRegression,0.999580,0.999856,0.990868,0.999711,0.995413
1,DecisionTreeClassifier,0.997201,0.998412,0.958333,0.998701,0.949541
2,RandomForestClassifier,0.999440,0.999856,0.986364,0.999567,0.995413
3,LGBMClassifier,0.999160,0.999567,0.986239,0.999567,0.986239
4,XGBClassifier,0.999160,0.999567,0.986239,0.999567,0.986239
5,GradientBoostingClassifier,0.998880,0.999423,0.981651,0.999423,0.981651
6,KNeighborsClassifier,0.997621,0.999855,0.931330,0.997690,0.995413
7,SVC,0.996501,0.997119,0.975369,0.999278,0.908257


Olhando os dados da tabela 3, concluímos que, para o Problema 3, todos os modelos tiveram excelente desemepenho, em todas as métricas. Não poderíamos pedir por algo melhor. 

## Problema 4: Prevendo se o composto é do grupo de controle

Agora estamos interessado em outro tipo de situação. Imagine que por um erro alguns dos rótulos dos compostos foram perdidos. Agora não se sabe se o composto era do grupo de controle ou não. Gostaríamos de poder prever essa informação. 
Para esse problema, imagino que estamos interessados tanto em baixa proporção de falsos positivos quanto em baixa proporção de falsos negativos. 

Para realizarmos isso, vamos voltar ao nosso data frame df_total. Nossa variável alvo será 'Eh controle' e a máquina preditiva irá receber as expressões gênicas, a viabilidade celular, o tempo do experimento e as doses.

In [ ]:
df_total

,id,tratamento,tempo,dose,composto,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18,g19,g20,g21,g22,g23,g24,g25,g26,g27,g28,g29,g30,g31,g32,g33,g34,...,c67,c68,c69,c70,c71,c72,c73,c74,c75,c76,c77,c78,c79,c80,c81,c82,c83,c84,c85,c86,c87,c88,c89,c90,c91,c92,c93,c94,c95,c96,c97,c98,c99,n_moa,ativo_moa,Eh controle,Tempo eh 24,Tempo eh 48,Tempo eh 72,ativa_2moa
0,id_000644bb2,com_droga,24,D1,b68db1d53,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,1.1830,0.1530,0.5574,-0.4015,0.1789,-0.6528,-0.7969,0.6342,0.1778,-0.3694,-0.5688,-1.1360,-1.1880,0.6940,0.4393,0.2664,0.1907,0.1628,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.3010,-0.1537,...,1.2570,-0.5979,1.2250,-0.0553,0.7351,0.5810,0.9590,0.2427,0.0495,0.4141,0.8432,0.6162,-0.7318,1.2120,0.6362,-0.4427,0.1288,1.4840,0.1799,0.5367,-0.1111,-1.0120,0.6685,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176,1,1,0,1,0,0,0
1,id_000779bfc,com_droga,72,D1,df89a8e5a,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,-1.1520,-0.4201,-0.0958,0.4590,0.0803,0.2250,0.5293,0.2839,-0.3494,0.2883,0.9449,-0.1646,-0.2657,-0.3372,0.3135,-0.4316,0.4773,0.2075,-0.4216,-0.1161,-0.0499,-0.2627,0.9959,-0.2483,0.2655,...,-0.3104,-0.6373,0.2887,-0.0765,0.2539,0.4443,0.5932,0.2031,0.7639,0.5499,-0.3322,-0.0977,0.4329,-0.2782,0.7827,0.5934,0.3402,0.1499,0.4420,0.9366,0.8193,-0.4236,0.3192,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371,0,0,0,0,0,1,0
2,id_000a6266a,com_droga,48,D1,18bb41b2c,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,-0.4797,-0.5631,-0.0366,-1.8300,0.6057,-0.3278,0.6042,-0.3075,-0.1147,-0.0570,-0.0799,-0.8181,-1.5320,0.2307,0.4901,0.4780,-1.3970,4.6240,-0.0437,1.2870,-1.8530,0.6069,0.4290,0.1783,0.0018,...,0.1210,-0.5177,-0.0604,0.1682,-0.4436,0.4963,0.1363,0.3335,0.9760,-0.0427,-0.1235,0.0959,0.0690,-0.9416,-0.7548,-0.1109,-0.6272,0.3019,0.1172,0.1093,-0.3113,0.3019,-0.0873,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931,3,1,0,0,1,0,1
3,id_0015fd391,com_droga,48,D1,8c7f86626,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,-0.8269,-0.3584,-0.8511,-0.5844,-2.5690,0.8183,-0.0532,-0.8554,0.1160,-2.3520,2.1200,-1.1580,-0.7191,-0.8004,-1.4670,-0.0107,-0.8995,0.2406,-0.2479,-1.0890,-0.7575,0.0881,-2.7370,0.8745,0.5787,...,-3.1660,0.2816,-0.2990,-1.1870,-0.5044,-1.7750,-1.6120,-0.9215,-1.0810,-3.0520,-3.4470,-2.7740,-1.8460,-0.5568,-3.3960,-2.9510,-1.1550,-3.2620,-1.5390,-2.4600,-0.9417,-1.5550,0.2431,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154,0,0,0,0,1,0,0
4,id_001626bd3,com_droga,72,D2,7cbed3131,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,-0.2219,-0.5121,-0.9577,1.1750,0.2042,0.1970,0.1244,-1.7090,-0.3543,-0.5160,-0.3330,-0.2685,0.7649,0.2057,1.3720,0.6835,0.8056,-0.3754,-1.2090,0.2965,-0.0712,0.6389,0.6674,-0.0783,1.1740,...,0.5178,0.5159,0.6091,0.1813,-0.4249,0.7832,0.6529,0.5648,0.4817,0.0587,0.5303,0.6376,-0.3966,-1.4950,-0.9625,-0.0541,0.6273,0.4563,0.0698,0.8134,0.1924,0.6054,-0.1824,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,com_droga,24,D2,df1d0a5a1,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,0.3055,-0.4726,0.1269,0.2531,0.1730,-0.4532,-1.0790,0.2474,-0.4550,0.3588,0.1600,-0.7362,-0.1103,0.8550,-0.4139,0.5541,0.2310,-0.5573,-0.4397,-0.9260,-0.2424,-0.6686,0.2326,0.6456,0.0136,-0.5141,-0.6320,0.7166,...,0.3154,-0.2071,-0.6158,-0.2977,0.0992,0.6838,0.5259,0.7882,0.3119,-0.7697,0.2203,-1.0710,0.5979,0.0848,-0.2555,0.6293,1.1660,0.3329,0.2754,0.410

Criando as variáveis:

In [ ]:
y = df_total['Eh controle']
x_aux = df_total.drop(['id', 'tratamento', 'tempo','dose', 'composto', 'ativo_moa', 'ativa_2moa', 'Eh controle'], axis = 1)
x = x_aux.drop(x_aux.columns[773:873], axis = 1)

Agora iremos separar nossos conjuntos de treino e conjunto de teste.

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, test_size = 0.3, stratify = y)

Como antes, vamos criar também um dataframe com informações sumarizadas das performaces de cada modelo.

In [ ]:
lista_modelo = []
lista_score = []
lista_precision0 = [] 
lista_precision1 = []
lista_recall0 = []
lista_recall1 = []

### Regressão Logistica

In [ ]:
modelo = LogisticRegression()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  95.0594821553534 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6417  168]
 [ 185  375]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      6585
           1       0.69      0.67      0.68       560

    accuracy                           0.95      7145
   macro avg       0.83      0.82      0.83      7145
weighted avg       0.95      0.95      0.95      7145



### DecisionTreeClassifier 

In [ ]:
modelo = DecisionTreeClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  88.91532540237928 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6169  416]
 [ 376  184]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      6585
           1       0.31      0.33      0.32       560

    accuracy                           0.89      7145
   macro avg       0.62      0.63      0.63      7145
weighted avg       0.89      0.89      0.89      7145



### RandomFlorestClassifier

In [ ]:
modelo = RandomForestClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  92.9741077676697 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6577    8]
 [ 494   66]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96      6585
           1       0.89      0.12      0.21       560

    accuracy                           0.93      7145
   macro avg       0.91      0.56      0.59      7145
weighted avg       0.93      0.93      0.90      7145



### LGBMClassifier

In [ ]:
modelo = LGBMClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  95.0594821553534 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6518   67]
 [ 286  274]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      6585
           1       0.80      0.49      0.61       560

    accuracy                           0.95      7145
   macro avg       0.88      0.74      0.79      7145
weighted avg       0.95      0.95      0.94      7145



### XGBClassifier

In [ ]:
modelo = XGBClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  94.61161651504548 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6517   68]
 [ 317  243]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      6585
           1       0.78      0.43      0.56       560

    accuracy                           0.95      7145
   macro avg       0.87      0.71      0.76      7145
weighted avg       0.94      0.95      0.94      7145



### GradientBoostingClassifier


In [ ]:
modelo = GradientBoostingClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  94.44366689993002 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6500   85]
 [ 312  248]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      6585
           1       0.74      0.44      0.56       560

    accuracy                           0.94      7145
   macro avg       0.85      0.71      0.76      7145
weighted avg       0.94      0.94      0.94      7145



### KNeighborsClassifier

In [ ]:
modelo = KNeighborsClassifier()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  93.65990202939119 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6519   66]
 [ 387  173]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.94      0.99      0.97      6585
           1       0.72      0.31      0.43       560

    accuracy                           0.94      7145
   macro avg       0.83      0.65      0.70      7145
weighted avg       0.93      0.94      0.92      7145



### Support Vector Machine

In [ ]:
modelo = SVC()
lista_modelo.append(str(modelo).split('(')[0])
modelo.fit(x_treino,y_treino)
resultado = modelo.score(x_teste,y_teste)
lista_score.append(resultado)
print('Score = ', resultado*100, '%')

Score =  95.52134359692093 %


In [ ]:
cm = confusion_matrix(y_teste, modelo.predict(x_teste))
lista_precision0.append(cm[0][0]/(cm[0][0]+cm[1][0]))
lista_precision1.append(cm[1][1]/(cm[1][1]+cm[0][1]))
lista_recall0.append(cm[0][0]/(cm[0][0]+cm[0][1]))
lista_recall1.append(cm[1][1]/(cm[1][1]+cm[1][0]))
print(cm)

[[6516   69]
 [ 251  309]]


In [ ]:
print(classification_report(y_teste, modelo.predict(x_teste)))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      6585
           1       0.82      0.55      0.66       560

    accuracy                           0.96      7145
   macro avg       0.89      0.77      0.82      7145
weighted avg       0.95      0.96      0.95      7145



### Resumo

In [ ]:
Tabela_Performace_Modelos = pd.DataFrame()
Tabela_Performace_Modelos['Modelo'] = lista_modelo
Tabela_Performace_Modelos['Score'] = lista_score
Tabela_Performace_Modelos['precision-0'] = lista_precision0
Tabela_Performace_Modelos['precision-1'] = lista_precision1
Tabela_Performace_Modelos['recall-0'] = lista_recall0
Tabela_Performace_Modelos['recall-1'] = lista_recall1

Tabela_Comparação_Modelos_Problema4 = Tabela_Performace_Modelos
Tabela_Comparação_Modelos_Problema4

,Modelo,Score,precision-0,precision-1,recall-0,recall-1
0,LogisticRegression,0.950595,0.971978,0.690608,0.974487,0.669643
1,DecisionTreeClassifier,0.889153,0.942552,0.306667,0.936826,0.328571
2,RandomForestClassifier,0.929741,0.930137,0.891892,0.998785,0.117857
3,LGBMClassifier,0.950595,0.957966,0.803519,0.989825,0.489286
4,XGBClassifier,0.946116,0.953614,0.781350,0.989674,0.433929
5,GradientBoostingClassifier,0.944437,0.954198,0.744745,0.987092,0.442857
6,KNeighborsClassifier,0.936599,0.943962,0.723849,0.989977,0.308929
7,SVC,0.955213,0.962908,0.817460,0.989522,0.551786


Estudando a tabela com o resumo das análises, percebemos que as acurácias estão altas, porém as outras méticas estão dispersas. Em geral, os modelos tem uma alta precisão para negativos, o que é razoável de se pensar, pois temos muito mais amostras que não são do grupo de controle. A precisão para positovos está boa, em geral (com excessão do decision tree), porém a revocação para positivos está bem baixa. Isso significa que há muitos do grupo de controle que estão sendo classficados como não sendo do grupo de controle. 
Minha conclusão é que os modelos tem desempenho mediano. 

## Outros problemas

Gostaria de sugerir outros problemas que podem ser atacados utilizando as mesmas téncinas de machine learning. Não os resolvo aqui por questão de tempo. Porém, me comprometo a estudar esses problemas nos próximos dias e então adicionar um outro arquivo no repositório, com as novas análises. 

### Prever se um composto ativa dois MOAs específicos.

Como a quantidade total de positivos será muito baixa (comparado com o total), espero uma acurácia alta, porém uma tendência à ter uma precisão baixa para positivos. 
Pretendo também testar como os modelos se saem com MOAs com alta correlação e MOAs com baixa correlação. 

### Prever se um composto ativa um MOA específico, mas não ativa outro MOA específico

Aqui espero um cenário similar ao do Problema 3, e portanto resultados similares. 

### Recuperar uma expressão gênica a partir dos MOAs ativados

Aqui estou propondo meio que o problema inverso do que estávamos fazendo. Não sei se faz sentido biológico ou não querer saber isso, mas achei interessante.
Obviamente, vou me concentrar em apenas uma expressão gênica (e.g., g0). 
Outro ponto a observar é que a natureza do problema é distinta. Até o momento, trabalhamos apenas com problemas de classficação binária. Este é um problema de regressão. Os modelos serão outros. 

### Recuperar o composto a partir das expressões gênicas

Agora trata-se um um problema de classficiação multi classe. Provavelmente será mais fácil se eu me restringir àqueles nove compostos mais frequentes. 

### Recuperar o composto a partir dos MOAs ativados

Similar ao problema anterior, com a vantagem de que penso ser mais fácil de se obter os dados em laboratório.
(quero dizer que penso que seja mais fácil verificar se um MOA foi ativado do que calcular a expressão gênica)